In [7]:
#%reset
## This code evaluates the performance (connectivity) of the network under extreme earthquake.
## Written by Mohammad Amin Nabian, mnabia2@illinois.edu, March 2017

## Import libraries
import numpy as np
import networkx as nx
import random
import copy
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn import cross_validation
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils, generic_utils
from keras.models import Sequential
from keras.models import model_from_json


## Initialization
nbridges=39
nlinks = 14
np.random.seed(777)

## Generating the network topology
G=nx.Graph()
G.add_node(0)
G.add_node(1)
G.add_node(2)
G.add_node(3)
G.add_node(4)
G.add_node(5)
G.add_node(6)
G.add_node(7)
G.add_node(8)
G.add_node(9)
G.add_node(10)
G.add_node(11)
G.add_edge(0,1)
G.add_edge(0,2)
G.add_edge(1,3)
G.add_edge(1,4)
G.add_edge(2,3)
G.add_edge(3,4)
G.add_edge(2,6)
G.add_edge(3,4)
G.add_edge(3,5)
G.add_edge(4,7)
G.add_edge(6,7)
G.add_edge(5,6)
G.add_edge(5,8)
G.add_edge(6,9)
G.add_edge(7,8)
G.add_edge(7,11)
G.add_edge(8,10)
G.add_edge(8,9)
G.add_edge(9,10)
G.add_edge(10,11)

## Assign the failure probabilities
s=(nlinks,2);b=np.zeros(s)
b[0]=(0,1)
b[1]=(0,2)
b[2]=(1,3)
b[3]=(1,4)
b[4]=(2,3)
b[5]=(2,6)
b[6]=(3,4)
b[7]=(3,5)
b[8]=(4,7)
b[9]=(5,6)
b[10]=(7,8)
b[11]=(7,11)
b[12]=(8,10)
b[13]=(9,10)

## load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Loaded model from disk")

## Evaluate the System Response

# Import the data for analysis and generate realizations
fpb = np.loadtxt('SURVIVALS_Evaluate.txt')
nsamplesEQ = np.shape(fpb)[1]
nsamplesMC = 1000

def link_failure(probs):
    new_probs = np.zeros(nlinks)
    new_probs[0] = np.log(probs[0])
    new_probs[1] = np.log(np.prod(probs[1:7]))
    new_probs[2] = np.log(probs[7])
    new_probs[3] = np.log(np.prod(probs[8:10]))
    new_probs[4] = np.log(np.prod(probs[10:16]))
    new_probs[5] = np.log(np.prod(probs[16:20]))
    new_probs[6] = np.log(np.prod(probs[20:23]))
    new_probs[7] = np.log(np.prod(probs[23:25]))
    new_probs[8] = np.log(probs[25])
    new_probs[9] = np.log(np.prod(probs[26:28]))
    new_probs[10] = np.log(probs[28])
    new_probs[11] = np.log(np.prod(probs[29:33]))
    new_probs[12] = np.log(np.prod(probs[33:35]))
    new_probs[13] = np.log(np.prod(probs[35:39]))
    return new_probs

fpb_multiply = np.zeros((nlinks,nsamplesEQ))
for i in range (nsamplesEQ):
    fpb_multiply[:,i] = link_failure(fpb[:,i])

fpb_samples = np.zeros((nlinks,nsamplesEQ,nsamplesMC),dtype=int)
for i in range (nlinks):
    for j in range (nsamplesEQ):
        for k in range (nsamplesMC):
            if (np.random.rand() < 1. - np.exp(fpb_multiply[i,j])):
                fpb_samples[i,j,k] = 1   # link will fail

# Calculate the network connectivity using ANN surrogate
def CONNECTIVITY_surrogate (index):
    class_result_flag = np.argmax(loaded_model.predict(fpb_samples[:,index,:].T),axis=-1)
    return class_result_flag
start_time = time.time()
class_result = Parallel(n_jobs=1)(delayed(CONNECTIVITY_surrogate)(index) for index in range(nsamplesEQ))
# Can't be done in parallel since keras uses all CPUs by default.
elapsed_time_surrogate = time.time() - start_time
print('The surrogate computational time is',elapsed_time_surrogate)
print('The mean connectivity by surrogate computation is',np.mean(class_result))

# Calculate the network connectivity using MCS
def CONNECTIVITY_standard (index):
    connect_flag = np.zeros(nsamplesMC)
    for p in range (nsamplesMC):
        T=copy.deepcopy(G)
        for i in range (nlinks):
            if fpb_samples[i,index,p] == 1:
                    T.remove_edge(*b[i])
        if nx.has_path(T,0,11):
            connect_flag[p] = 1
    del T
    return connect_flag

start_time = time.time()
connect_matrix = Parallel(n_jobs=8)(delayed(CONNECTIVITY_standard)(index) for index in range(nsamplesEQ))
elapsed_time_standard = time.time() - start_time
print('The standard computational time is',elapsed_time_standard)
print('The mean connectivity by standard computation is',np.mean(connect_matrix))
np.savetxt('connectivity_results_standard.txt',connect_matrix)
np.savetxt('connectivity_results_surrogate.txt',class_result)

# Reshape
fpb_samples_reshaped = np.zeros((nsamplesEQ*nsamplesMC,nlinks),dtype=int)
for i in range (nlinks):
    c = -1
    for j in range (nsamplesEQ):
        for k in range (nsamplesMC):
            c += 1
            fpb_samples_reshaped[c,i] = fpb_samples[i,j,k]
            
np.savetxt('samples_evaluate.txt',fpb_samples_reshaped)


Loaded model from disk
The surrogate computational time is 53.886054039001465
The mean connectivity by surrogate computation is 0.9000634
The standard computational time is 603.9692530632019
The mean connectivity by standard computation is 0.9001638
